In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta, time
from itertools import combinations 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
ordercustomer = pd.read_csv("order_customer.csv", index_col = 0)
orderproduct = pd.read_csv("product_amount.csv", index_col = 0)
empty = pd.read_csv("empty.csv", index_col = 0)
full = pd.read_csv("full.csv", index_col = 0)
warehouseframe = pd.read_csv("warehouse.csv", index_col = 0)
customer = pd.read_csv("customer.csv", index_col = 0)
productwarehouse = pd.read_csv("productwarehouse.csv", index_col = 0)
traveltime = pd.read_csv("traveltime.csv", index_col = 0)
orderday = pd.read_csv("orderday.csv", index_col = 0)
#series closest city for destination,
newempty = empty.idxmin(axis=1)

productwarehouse = productwarehouse[~productwarehouse.index.duplicated(keep='last')]

vehicletravelcols = ["Order Id(s)", "Vehicle Id", "Customer Id", "Departure Loc", "Departure Time", "Destination Loc", "Return Loc", "Return Time"]
travelcostcols = ["Cost"]
travelcostframe = pd.DataFrame(columns= travelcostcols)
vehicletravelframe = pd.DataFrame(columns=vehicletravelcols)

In [3]:
# otherPF returns the array of being in the warehouses of the other orders of the customer
def otherPF(droppedProduct, droppedCustomer, customerId):
    p1 = otherorders(droppedProduct,droppedCustomer,customerId)["Product"].values
    return productwarehouse.loc[p1].values

In [4]:
def combineWith(weightLimit, selectedPF, droppedProduct, droppedCustomer, customerId):
    
    beingArray = otherPF(droppedProduct, droppedCustomer, customerId)
    deletelist = list()
    weightArray = otherorders(droppedProduct, droppedCustomer, customerId)["Weight"].values
    weightArray = weightArray.reshape(len(weightArray),1)
    for i in range(0,len(weightArray)):
        if(weightArray[i]>weightLimit):
            deletelist.append(i)
            
    beingArray = np.delete(beingArray,tuple(deletelist),axis =0)
    weightArray = np.delete(weightArray, tuple(deletelist),axis =0)    
    beingArray = beingArray*selectedPF
    calculationArray = beingArray*weightArray
    calculationArray = calculationArray.T
    productArray = otherOrdersIndices(droppedProduct, droppedCustomer, customerId)
    productArray = np.delete(productArray, tuple(deletelist),axis=0)
    tempMax = 0
    tempI = 0
    indice = 0
    factorial = 0
    tempFactory = 0
    factoryCount = 0
    indexTuple = ()
    
    for k in calculationArray:
        for j in range(1,len(productArray)+1):
            for i in combinations(k, j):
                if(tempMax<sum(i)<=weightLimit):
                    tempMax = sum(i)
                    indice = tempI
                    factorial = j
                    factoryCount = tempFactory
                tempI+=1
            tempI=0
        tempFactory+=1
        
    indexTuple = list(combinations(productArray,factorial))[indice]
    if((tempMax)>0):
        return indexTuple , factoryCount, tempMax
    else:
        return tuple(), 0, 0


In [5]:
#Returns other orders of the customer with given customerId
def otherOrdersIndices(droppedProduct, droppedOrder, customerId):
    return (droppedProduct.loc[incrementlist(
        convertlist((
            (droppedOrder.loc[droppedOrder["Customer Id"]==customerId]).index).tolist()), -1)].index.values)

In [6]:
def otherorders(droppedProduct, droppedOrder, customerId):
    indexlist = otherOrdersIndices(droppedProduct, droppedOrder, customerId)
    return droppedProduct.loc[indexlist]

In [7]:
cityplate = {}
cityplate["1"] = "A"
cityplate["2"] = "B"
cityplate["3"] = "C"
cityplate["4"] = "D"
cityplate["A"] = 1
cityplate["B"] = 2
cityplate["C"] = 3
cityplate["D"] = 4

def calctime(dep, dest):
    if type(dest) == int:
        dest = str(dest)
    dest = str(cityplate[dest])
    return traveltime.loc[dep][dest]

In [8]:
def get_minweight(pframe, ocframe, customer_id):
    minlist = pframe.loc[incrementlist(convertlist(
            ((ocframe.loc[ocframe["Customer Id"]==customer_id]).index).tolist()),
                                                   -1)]["Weight"].tolist()
    try:
        return min(minlist) , minlist
    except:
        return 26000, []

def convertlist(List):
    
    newlist = list()
    for x in List:
        newlist.append(int(re.search(r'\d+', x).group()))
    return newlist

def incrementlist(List, value):
    
    newlist = list(map(lambda x: x + value, List))
    return newlist


def calculatefullcost(departure, destination):
    if type(departure) == str:
        if departure == "A":
            cost = full.loc[destination]["1"]
        elif departure == "B":
            cost = full.loc[destination]["2"]
        elif departure == "C":
            cost = full.loc[destination]["3"]
        elif departure == "D":
            cost = full.loc[destination]["4"]
        else:
            raise "No such Warehouse!!!\n"
    else:
        cost = full.loc[destination][str(departure)]
    return cost

def calculateemptycost(departure, destination="", closest=1):
    
    cost = empty.loc[departure][str(newempty[departure])]
    
    if (closest):
        return cost, cityplate[str(newempty[departure])]
    else:
        if type(destination) == str:
            if destination == "A":
                cost = empty.loc[departure]["1"]
            elif destination == "B":
                cost = empty.loc[departure]["2"]
            elif destination == "C":
                cost = empty.loc[departure]["3"]
            elif destination == "D":
                cost = empty.loc[departure]["4"]
            else:
                raise Exception("No such warehouse")
            plate = destination
        else:
            cost = empty.loc[departure][str(destination)]
            plate = cityplate[str(destination)]
        return cost, plate

In [9]:
def indiceToOrder(List):
    if(type(List) == list):
        newList = list()
        for x in List:
            newList.append("Order {}".format(str(x+1)))
        return newList
    elif(type(List)==int):
        return "Order {}".format(str(List+1))

In [10]:
class warehouse:
    def __init__(self, name, cif, vehiclelimit):
        
        self.vehiclelimit = vehiclelimit
        self.name = name
        self.listofvehicles = list()
        self.cif = cif
        self.poppedvehicle = 0
        self.numberofvehicles = 0
        
    def append(self, vehicle):
        
        self.numberofvehicles += 1
        self.listofvehicles.append(vehicle)
        vehicle.city = self.name
        self.sortvehicles()
        
    def sortvehicles(self):
        self.listofvehicles = sorted(self.listofvehicles, key=lambda x: (x.available_at, -x.isDiscounted))
    #change day 
    def change_day(self, c_date):
        for element in self.listofvehicles:
            dtime = element.available_at
            if dtime.date() < c_date:
                element.available_at= element.available_at.replace(day=element.available_at.date().day + 1, hour = 8, minute=0, microsecond = 0)
            
    #check if factory is available for sending the vehicle
    def available(self):
        return (self.cif > self.poppedvehicle)
    
    def reset_pop(self):
        self.poppedvehicle = 0
        
    def __str__(self):
        return "Warehouse: " + self.name + "\n"
    
    def __repr__(self):
        return self.__str__()
        
    def pop(self):
        try:
            v = self.listofvehicles.pop(0)
            self.poppedvehicle += 1
            self.numberofvehicles = self.numberofvehicles -1 
            return v
        except Exception as e:
            print("Warehouse: {} is empty".format(self.name))
    
    def isvehicleneeded(self):
        return self.vehiclelimit>self.numberofvehicles
    
            
                
                

In [11]:
class vehicle:
    def __init__(self, index, city, dateobj = datetime(2019, 5, 1, 8, 0) , isDiscounted = 0):
        self.index = index
        self.city = city
        self.isDiscounted = isDiscounted
        self.available_at = dateobj
        self.orders = list()
        
        
    def get_available(self):
        return self.available_at
    
    def set_available(self, timed):
        delta = timedelta(hours= timed)
        self.available_at = self.available_at + delta
        if(self.available_at.time() < time(8)):
            self.available_at = self.available_at.replace(hour=8,minute=0,second=0, microsecond=0)
        elif(self.available_at.time()>time(18)):
            self.available_at = self.available_at + timedelta(days=1)
            self.available_at = self.available_at.replace(hour=8,minute=0,second=0, microsecond=0)
            
    def __str__(self):
        return "Vehicle: " + str(self.index) 

    def __repr__(self):
        return self.__str__()

In [12]:
warehousedict = dict()
warehousedict["A"] = warehouse("A", int(warehouseframe.loc["A"]["CIF.1"]), 50)
warehousedict["B"] = warehouse("B", int(warehouseframe.loc["B"]["CIF.1"]), 50)
warehousedict["C"] = warehouse("C", int(warehouseframe.loc["C"]["CIF.1"]), 50)
warehousedict["D"] = warehouse("D", int(warehouseframe.loc["D"]["CIF.1"]), 50)


In [13]:
#number of vehicles rented
rented = 130
#number of vehicles owned
owned = 12
startdate = datetime(2019, 5, 1, 8, 0)
#initializing vehicles
for i in range(0,owned+rented):
    if i < owned + 1:
        if i%4 == 0:
            warehousedict["A"].append(vehicle(i, "A", startdate, isDiscounted = 1))
        elif i%4 == 1:
            warehousedict["B"].append(vehicle(i, "B", startdate, isDiscounted = 1))
        elif i%4 == 2:
            warehousedict["C"].append(vehicle(i, "C", startdate, isDiscounted = 1))
        else:
            warehousedict["D"].append(vehicle(i, "D", startdate, isDiscounted = 1))
    else:
        if i%4 == 0:
            warehousedict["A"].append(vehicle(i, "A", startdate, isDiscounted = 0))
        elif i%4 == 1:
            warehousedict["B"].append(vehicle(i, "B", startdate, isDiscounted = 0))
        elif i%4 == 2:
            warehousedict["C"].append(vehicle(i, "C", startdate, isDiscounted = 0))
        else:
            warehousedict["D"].append(vehicle(i, "D", startdate, isDiscounted = 0))


minday = orderday.min(1).min()
maxday = orderday.max(1).max()

In [14]:
tempdeparture = ""
platecode = 0
customerid = 0
orderid = 0
product = ""
where = ""
tempmincost = 99999

vframecounter = 0
shiptime = 0
returntime = 0
vehicletime = 0

decreasingordercustomer = ordercustomer.copy()
decreasingday = orderday.copy()
decreasingproduct= orderproduct.reset_index(level=0, inplace=False)
for i in range(minday, maxday+1):
    #orders of i'th day
    order_round = decreasingday.loc[decreasingday["Day"]==i]
    todayordercustomer = decreasingordercustomer.loc[indiceToOrder(decreasingday.loc[decreasingday["Day"]==i].index.tolist())]
    #product weight frame of the i'th day
    todayproduct = decreasingproduct.loc[decreasingday.loc[decreasingday["Day"]==i].index]
    for index, row in order_round.iterrows():
        vehicletime = 0
        shiptime = 0
        returntime = 0
        
        if(not(indiceToOrder(index) in todayordercustomer.index)):
            continue
        #get order id
        orderid = int(re.search(r'\d+', row["Order No"]).group())
        #get customer id
        customerid = todayordercustomer.loc["Order {}".format(str(orderid))]["Customer Id"]
        #get plate code
        platecode = customer.loc[customerid]["Plate Code"].astype(int)
        #product id
        product = todayproduct.loc[index].Product
        #weight of order
        weight = todayproduct.loc[index].Weight

        if weight >= 22000:
            #droptheorder from order_round and orderday temp list
            decreasingday = decreasingday.drop(index)
            todayordercustomer = todayordercustomer.drop(indiceToOrder(index))
            todayproduct = todayproduct.drop(index)
            #drop order from temp ordercustomer table
            decreasingordercustomer = decreasingordercustomer.drop(indiceToOrder(index))
            #drop order from temp orderproduct table
            decreasingproduct = decreasingproduct.drop(index)
            #calculate the minimum weight of the same customer's orders for comparison
            minweight, minlist = get_minweight(todayproduct, todayordercustomer, customerid)
            
            #try to combine
            if (weight + minweight <= 26000):
                tempmincost = 999999
                #weightlimit for appending other weights
                weightLim = 26000-weight
                beingArray = productwarehouse.loc[product].values

                ## combineWith returns the combination with other orders and departure factory.
                other_products, depart_factory , _= combineWith(weightLim,beingArray,todayproduct,todayordercustomer,customerid)
                #unable to combine with other orders
                if(len(other_products)==0):
                    tempmincost = 9999999
                    isSent = 0
                    for colname in productwarehouse.columns:
                        if((productwarehouse.loc[product][colname]) == 1 and warehousedict[colname].available()):
                            isSent = 1
                            cost_c = calculatefullcost(colname, platecode)
                            if(cost_c < tempmincost):
                                tempmincost = cost_c
                                tempdeparture = colname
                    #able to ship the order
                    if(isSent == 1):
                        ordervehicle = warehousedict[tempdeparture].pop()
                        cost_c, destwar = calculateemptycost(platecode)
                        if(warehousedict[destwar].isvehicleneeded()):
                            tempmincost = tempmincost + cost_c

                        else:
                            cost_c = 9999999
                            #search for other alternatives with minimum cost
                            for colname in productwarehouse.columns:
                                if(destwar!=colname and warehousedict[colname].isvehicleneeded()):
                                    if(cost_c > calculateemptycost(platecode,colname,0)[0]):
                                        cost_c, temp_dest = calculateemptycost(platecode, colname, 0)
                            destwar =temp_dest
                            tempmincost = tempmincost + cost_c
                        #cost calculation
                        if ordervehicle.isDiscounted == 1:
                            tempmincost = tempmincost - (tempmincost/10)
                        #time delta calculation
                        shiptime = calctime(platecode,tempdeparture)
                        returntime = calctime(platecode,destwar)
                        vehicletime = ordervehicle.available_at
                        vehicletravelframe.loc[vframecounter] = ["{}".format(orderid), ordervehicle.index, customerid,tempdeparture,vehicletime,platecode,destwar,vehicletime+timedelta(hours=(4+returntime+shiptime))]
                        travelcostframe.loc[vframecounter] = tempmincost
                        vframecounter = vframecounter +1
                        #set new available time
                        ordervehicle.set_available(4+returntime+shiptime)
                        warehousedict[temp_dest].append(ordervehicle)
                        
                    #unable to ship the order
                    else:
                        decreasingday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                        orderday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                        decreasingday = decreasingday.sort_index()
                        #print("Couldn't ship order: No", orderid)
                        
                #able to combine with other orders                    
                else:
                    
                    if(warehousedict[productwarehouse.columns[depart_factory]].available()):
                        o_list = list()
                        for element in other_products:
                            element = (int)(element)
                            o_list.append(todayproduct.loc[element]["Product"])
                            decreasingday = decreasingday.drop(element)
                            todayordercustomer = todayordercustomer.drop(indiceToOrder(element))
                            todayproduct = todayproduct.drop(element)
                            #drop order from temp ordercustomer table
                            decreasingordercustomer = decreasingordercustomer.drop(indiceToOrder(element))
                            #drop order from temp orderproduct table
                            decreasingproduct = decreasingproduct.drop(element)

                        tempmincost = calculatefullcost(productwarehouse.columns[depart_factory], platecode)

                        ordervehicle = warehousedict[productwarehouse.columns[depart_factory]].pop()
                        cost_c, destwar = calculateemptycost(platecode)

                        if(warehousedict[destwar].isvehicleneeded()):
                            tempmincost = tempmincost + cost_c
                        else:
                            cost_c = 99999999
                            for colname in productwarehouse.columns:
                                if(destwar!=colname and warehousedict[colname].isvehicleneeded()):
                                    if(cost_c > calculateemptycost(platecode,colname,0)[0]):
                                        cost_c, temp_dest = calculateemptycost(platecode, colname,0)

                            tempmincost = tempmincost + cost_c
                            destwar = temp_dest
                        #cost calculation
                        if ordervehicle.isDiscounted == 1:
                            tempmincost = tempmincost - (tempmincost/10)
                        #delta time calculation
                        shiptime = calctime(platecode,productwarehouse.columns[depart_factory])
                        returntime = calctime(platecode,destwar)
                        vehicletime = ordervehicle.available_at
                        vehicletravelframe.loc[vframecounter] = ["{}, {}".format(orderid,incrementlist(list(other_products),1)), ordervehicle.index, customerid,productwarehouse.columns[depart_factory],vehicletime,platecode,destwar,vehicletime+timedelta(hours=(4+returntime+shiptime))]
                        travelcostframe.loc[vframecounter] = tempmincost
                        ordervehicle.set_available(4+returntime+shiptime)
                        vframecounter = vframecounter +1
                        warehousedict[destwar].append(ordervehicle)
                    else:
                        decreasingday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                        orderday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                        decreasingday = decreasingday.sort_index()
            #send directly
            else:
                tempmincost = 999999
                isSent = 0
                for colname in productwarehouse.columns:
                    #check if product exists in warehouse "colname" and cif value of warehouse
                    if ((productwarehouse.loc[product][colname]==1) and warehousedict[colname].available()):
                        #calculate the cost from that warehouse
                        isSent = 1
                        cost_c = calculatefullcost(colname, platecode)
                        
                        if(cost_c < tempmincost):
                            tempmincost = cost_c
                            tempdeparture = colname
                if(isSent == 0):
                    decreasingday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                    orderday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                    decreasingday = decreasingday.sort_index()
                    continue
                ordervehicle = warehousedict[tempdeparture].pop()
                cost_c, destwar = calculateemptycost(platecode)
                
                if(warehousedict[destwar].isvehicleneeded()):
                    tempmincost = tempmincost + cost_c
                else:
                    cost_c = 999999
                    for colname in productwarehouse.columns:
                        if(destwar != colname):
                            if(cost_c > calculateemptycost(platecode, colname, 0)[0]):
                                cost_c, temp_dest = calculateemptycost(platecode, colname, 0)
                    destwar = temp_dest
                    tempmincost = tempmincost + cost_c
                    
                #cost calculation
                if ordervehicle.isDiscounted == 1:
                    tempmincost = tempmincost - (tempmincost/10)
                #delta time calculation
                shiptime = calctime(platecode,tempdeparture)
                returntime = calctime(platecode,destwar)
                vehicletime = ordervehicle.available_at
                vehicletravelframe.loc[vframecounter] = ["{}".format(orderid), ordervehicle.index, customerid,tempdeparture,vehicletime,platecode,destwar,vehicletime+timedelta(hours=(4+returntime+shiptime))]
                
                travelcostframe.loc[vframecounter] = tempmincost
                vframecounter = vframecounter +1 
                ordervehicle.set_available(4+returntime+shiptime)
                warehousedict[destwar].append(ordervehicle)
                
        #send with combination
        else:
            decreasingday = decreasingday.drop(index)
            todayproduct = todayproduct.drop(index)
            todayordercustomer = todayordercustomer.drop(indiceToOrder(index))
            weightLim = 26000 - weight
            beingArray = productwarehouse.loc[product].values
            other_products, depart_factory, temp_weight = combineWith(weightLim,beingArray,todayproduct,todayordercustomer,customerid)
            
            if(temp_weight+weight<22000 or not warehousedict[productwarehouse.columns[depart_factory]].available()):
                decreasingday.loc[index] = [("Order {}".format(str(index+1))),i+1]
                decreasingday = decreasingday.sort_index()
            else:
                tempmincost = calculatefullcost(productwarehouse.columns[depart_factory], platecode)
                ordervehicle = warehousedict[productwarehouse.columns[depart_factory]].pop()
                cost_c, destwar = calculateemptycost(platecode)

                if(warehousedict[destwar].isvehicleneeded()):
                    tempmincost = tempmincost + cost_c
                else:
                    cost_c = 99999999
                    for colname in productwarehouse.columns:
                        if(destwar!=colname and warehousedict[colname].isvehicleneeded()):
                            if(cost_c > calculateemptycost(platecode,colname,0)[0]):
                                cost_c, temp_dest = calculateemptycost(platecode, colname,0)
                    destwar = temp_dest
                    tempmincost = tempmincost + cost_c
                    
                #cost calculation
                if ordervehicle.isDiscounted == 1:
                    tempmincost = tempmincost - (tempmincost/10)
                ##delta time calculation
                shiptime = calctime(platecode,productwarehouse.columns[depart_factory])
                returntime = calctime(platecode,destwar)
                vehicletime = ordervehicle.available_at
                vehicletravelframe.loc[vframecounter] = ["{}, {}".format(orderid,incrementlist(list(other_products),1)), ordervehicle.index, customerid,productwarehouse.columns[depart_factory],vehicletime,platecode,destwar,vehicletime+timedelta(hours=(4+returntime+shiptime))]

                ordervehicle.set_available(returntime+shiptime+4)            

                warehousedict[destwar].append(ordervehicle)
                travelcostframe.loc[vframecounter] = tempmincost
                vframecounter = vframecounter +1 
                #delete sent items
                o_list = list()
                for element in other_products:
                    element = (int)(element)
                    o_list.append(todayproduct.loc[element]["Product"])
                    decreasingday = decreasingday.drop(element)
                    todayordercustomer = todayordercustomer.drop(indiceToOrder(element))
                    todayproduct = todayproduct.drop(element)
                    #drop order from temp ordercustomer table
                    decreasingordercustomer = decreasingordercustomer.drop(indiceToOrder(element))
                    #drop order from temp orderproduct table
                    decreasingproduct = decreasingproduct.drop(element)
                
    startdate = startdate + timedelta(days=1)
    for key, value in warehousedict.items():
        value.reset_pop()
        value.change_day(startdate.date())

In [15]:
vehicletravelframe

,Order Id(s),Vehicle Id,Customer Id,Departure Loc,Departure Time,Destination Loc,Return Loc,Return Time
0,1,3,100050,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527
1,2,7,100050,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527
2,3,1,100062,Bursa,2019-05-01 08:00:00.000000,39,Bursa,2019-05-02 03:00:00.000000
3,4,0,100223,Kirklareli,2019-05-01 08:00:00.000000,41,Bursa,2019-05-01 19:10:25.714286
4,5,4,100223,Kirklareli,2019-05-01 08:00:00.000000,41,Bursa,2019-05-01 19:10:25.714286
5,6,5,100674,Bursa,2019-05-01 08:00:00.000000,7,Ankara,2019-05-02 06:13:36.000001
6,10,11,101678,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527
7,11,13,101678,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527
8,12,17,101678,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527
9,13,21,101678,Ankara,2019-05-01 08:00:00.000000,34,Kirklareli,2019-05-01 22:48:44.210527


In [16]:
print(travelcostframe.sum())

Cost    517160.494152
dtype: float64


In [17]:
travelcostframe.to_excel("travelcostframe.xls")
vehicletravelframe.to_excel("vehicletravelframe.xls")